In [41]:
from spacy.util import filter_spans
from tqdm import tqdm
import spacy
from spacy.training.example import Example #Müssen später vermutlich was anderes nehmen
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import os
from training_set import training_data_got

import spacy
 
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
 
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [42]:
training_data = []
for example in tqdm(training_data_got[1]):
    text, annotations = example[0], example[1]

    entities = annotations['entities']
    doc = nlp.make_doc(text)
    spans = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            spans.append(span)
    filtered_spans = filter_spans(spans)
    doc.ents = filtered_spans
    training_data.append((doc, annotations))


100%|██████████| 1/1 [00:00<?, ?it/s]


In [43]:
doc_bin = spacy.tokens.DocBin()
for doc, annotations in training_data:
    doc_bin.add(doc, annotations=annotations)
doc_bin.to_disk("training_data.spacy")


TypeError: DocBin.add() got an unexpected keyword argument 'annotations'

In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0
